# Reductions in memory

In this benchmark we will compare ironArray's performance with Zarr, HDF5, Numpy and TileDB. We will work with the data used in the [Reductions tutorial](../tutorials/05.Reductions.html) with the default chunk shapes for every library and compute the same reduction for every case with in memory operands.  It is important to stress that we are using defaults for every library, which are typically fine tuned for general performance; this is a pretty fair way to compare them without spending long time optimizing for every one.


Let's go:

In [1]:
%load_ext memprofiler

import numpy as np
import iarray as ia
import os
import zarr
import dask.array as da
import h5py
import hdf5plugin as h5plugin
from numcodecs import Blosc
import tiledb

In [2]:
!du -sh ../tutorials/precip-3m.iarr

672M	../tutorials/precip-3m.iarr


### ironArray

In [3]:
%%time

ia_precip = ia.load("../tutorials/precip-3m.iarr")
print(ia_precip.shape)
print("cratio: ", round(ia_precip.cratio, 2))
chunks = ia_precip.chunks
print(ia_precip.info)

(3, 720, 721, 1440)
cratio:  15.43
type   : IArray
shape  : (3, 720, 721, 1440)
chunks : (1, 128, 128, 256)
blocks : (1, 16, 32, 64)
cratio : 15.43

CPU times: user 16.9 ms, sys: 475 ms, total: 492 ms
Wall time: 785 ms


In [4]:
%%mprof_run 1.iarray::std_memory
ia_reduc = ia.std(ia_precip, axis=(3, 0, 2)).data

memprofiler: used 3.77 MiB RAM (peak of 8.60 MiB) in 18.0556 s, total RAM usage 822.28 MiB


### NumPy

Let's do the same computation with NumPy. First we will get the NumPy array from the ironArray one:

In [5]:
%%time
precip = ia_precip.data

CPU times: user 8.1 s, sys: 3.66 s, total: 11.8 s
Wall time: 8.41 s


and now the actual reduction:

In [6]:
%%mprof_run 5.numpy::std_memory
np_reduc = np.std(precip, axis=(3, 0, 2))

memprofiler: used -7246.96 MiB RAM (peak of 1540.45 MiB) in 49.5041 s, total RAM usage 36.92 MiB


In [7]:
np.testing.assert_allclose(ia_reduc, np_reduc, atol=1e-5, rtol=1e-5)
del precip
del np_reduc

### Zarr

In [8]:
precip_zarr = zarr.array(ia_precip.data)
print(precip_zarr.info)
precip_zdask = da.from_zarr(precip_zarr)

Type               : zarr.core.Array
Data type          : float32
Shape              : (3, 720, 721, 1440)
Chunk shape        : (1, 90, 91, 180)
Order              : C
Read-only          : False
Compressor         : Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Store type         : builtins.dict
No. bytes          : 8970393600 (8.4G)
No. bytes stored   : 1117920644 (1.0G)
Storage ratio      : 8.0
Chunks initialized : 1536/1536



In [9]:
%%mprof_run 2.zarr::std_memory
zdask_reduc = da.std(precip_zdask, axis=(3, 0, 2))
zarr_reduc = zarr.create(shape=ia_reduc.shape, dtype=ia_reduc.dtype)
da.to_zarr(zdask_reduc, zarr_reduc)

memprofiler: used 126.86 MiB RAM (peak of 8117.93 MiB) in 7.0446 s, total RAM usage 1220.31 MiB


In [10]:
np.testing.assert_almost_equal(zarr_reduc, ia_reduc)
del precip_zdask
del zdask_reduc
del zarr_reduc

AssertionError: 
Arrays are not almost equal to 7 decimals

Mismatched elements: 713 / 720 (99%)
Max absolute difference: 5.718088e-06
Max relative difference: 0.0162207
 x: array([0.0003691, 0.0003663, 0.0003644, 0.0003662, 0.0003712, 0.0003774,
       0.0003794, 0.0003722, 0.0003795, 0.0003822, 0.0003826, 0.0003876,
       0.0003919, 0.0003969, 0.000399 , 0.000399 , 0.0004011, 0.0004044,...
 y: array([0.0003663, 0.0003637, 0.0003621, 0.0003642, 0.0003688, 0.0003746,
       0.0003761, 0.0003693, 0.000376 , 0.0003786, 0.0003791, 0.0003839,
       0.0003882, 0.0003929, 0.0003949, 0.0003949, 0.0003969, 0.0004001,...

### HDF5

In [ ]:
h5_urlpath = "precip-3m.hdf5"

if not os.path.exists(h5_urlpath):
    with h5py.File(h5_urlpath, "w") as f:
        h5_precip = f.create_dataset("h5_precip", ia_precip.shape, dtype=ia_precip.dtype, **h5plugin.Blosc())
        ia_precip.copyto(h5_precip)


h5_file = h5py.File(h5_urlpath, "r", driver='core', backing_store=False)
precip_h5 = h5_file['h5_precip']


precip_h5dask = da.from_array(precip_h5)

h5_reduc_urlpath = "reduc.hdf5"
ia.remove_urlpath(h5_reduc_urlpath)

In [ ]:
%%mprof_run 3.hdf5::std_memory

h5dask_reduc = da.std(precip_h5dask, axis=(3, 0, 2))

f = h5py.File(h5_reduc_urlpath, "w", driver='core', backing_store=False)
h5_reduc = f.create_dataset(name=h5_reduc_urlpath, shape=ia_reduc.shape, dtype=ia_reduc.dtype, **h5plugin.Blosc())
da.to_hdf5(h5_reduc_urlpath, '/x', h5dask_reduc)

In [ ]:
del h5dask_reduc
del precip_h5dask
del h5_reduc

### TileDB

In [ ]:
# TileDB does not come with defaults for chunk shape (tiles), so using ironArray's one
shape = ia_precip.shape
adom = tiledb.Domain(
        tiledb.Dim(name="rows", domain=(0, shape[0] - 1), dtype=np.int32, tile=chunks[0]),
        tiledb.Dim(name="cols", domain=(0, shape[1] - 1), dtype=np.int32, tile=chunks[1]),
        tiledb.Dim(name="else", domain=(0, shape[2] - 1), dtype=np.int32, tile=chunks[2]),
        tiledb.Dim(name="else2", domain=(0, shape[3] - 1), dtype=np.int32, tile=chunks[3]),
    )

filters = tiledb.FilterList([tiledb.ByteShuffleFilter(), tiledb.LZ4Filter(5)])
aschema = tiledb.ArraySchema(
    domain=adom,  sparse=False, attrs=[tiledb.Attr(name="a", dtype=np.float32, filters=filters)]
)

# Create the (empty) array on disk.
tiledb_name = "mem://precip-3m-optimal.tiledb"
if not os.path.exists(tiledb_name):
    tiledb.DenseArray.create(tiledb_name, aschema)
    with tiledb.DenseArray(tiledb_name, mode="w") as A_opt:
        A_opt[:] = ia_precip.data

dask_tiledb = da.from_tiledb(tiledb_name, attribute='a')

In [ ]:
%%mprof_run 4.tiledb::std_memory
res_dask_opt = da.std(dask_tiledb, axis=(3, 0, 2))
res_dask_opt.to_tiledb("mem://res_tiledb")

In [ ]:
del res_dask_opt
del dask_tiledb
tiledb.remove(tiledb_name)
tiledb.remove("mem://res_tiledb")

### Results

Finally, we will do a plot of the memory and time consumption with each different library.

In [ ]:
%mprof_plot .*::std

So in this case, Zarr, HDF5 and TileDB consume lots of memory, whereas ironArray usage is really meager. NumPy consumes less memory (actually, no memory is consumed) than ironArray. And when it comes to speed ironArray performance is the best in its class, only surpassed by NumPy (but not by a large extent).